In [12]:
import sys
print(f"Python version: {sys.version}")
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

Python version: 3.9.4 (tags/v3.9.4:1f2e308, Apr  6 2021, 13:40:21) [MSC v.1928 64 bit (AMD64)]


In [13]:
train_data = pd.read_csv('chf_train.csv')
valid_data = pd.read_csv('chf_valid.csv')

# Split inputs (X) and outputs (y)
x_train = train_data.iloc[:, :6].values
y_train = train_data.iloc[:, 6].values.reshape(-1, 1)
x_test = valid_data.iloc[:, :6].values
y_test = valid_data.iloc[:, 6].values.reshape(-1, 1)

# Scale data
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

X_train = x_scaler.fit_transform(x_train)
X_test = x_scaler.transform(x_test)
Y_train = y_scaler.fit_transform(y_train)
Y_test = y_scaler.transform(y_test)

In [14]:
model = Sequential([
    Dense(300, activation='relu', input_shape=(6,)),
    Dense(300, activation='relu'),
    Dense(300, activation='relu'),
    Dense(1, activation='linear')
])

# Compile model
model.compile(optimizer=Adam(learning_rate=1e-3), loss='mse')

In [15]:
history = model.fit(X_train, Y_train, epochs=100, batch_size=8, verbose=1, validation_data=(X_test, Y_test))

Epoch 1/100
2012/2012 [==============================] - 9s 4ms/step - loss: 9.9489e-04 - val_loss: 3.5637e-04
Epoch 2/100
2012/2012 [==============================] - 5s 2ms/step - loss: 4.2013e-04 - val_loss: 3.1072e-04
Epoch 3/100
2012/2012 [==============================] - 4s 2ms/step - loss: 3.2054e-04 - val_loss: 1.8117e-04
Epoch 4/100
2012/2012 [==============================] - 4s 2ms/step - loss: 2.8508e-04 - val_loss: 2.0472e-04
Epoch 5/100
2012/2012 [==============================] - 4s 2ms/step - loss: 2.2394e-04 - val_loss: 1.1691e-04
Epoch 6/100
2012/2012 [==============================] - 5s 2ms/step - loss: 2.0616e-04 - val_loss: 8.0310e-05
Epoch 7/100
2012/2012 [==============================] - 5s 2ms/step - loss: 1.9409e-04 - val_loss: 1.1181e-04
Epoch 8/100
2012/2012 [==============================] - 5s 2ms/step - loss: 1.7844e-04 - val_loss: 3.3561e-04
Epoch 9/100
2012/2012 [==============================] - 5s 2ms/step - loss: 1.5395e-04 - val_loss: 1.2261e-04
E

In [16]:
import pandas as pd
import numpy as np

# Assuming X_test, Y_test, and the model are already defined
# Predict the output using the model
Y_test_pred = model.predict(X_test)

# Combine X_test, Y_test, and Y_test_pred into a single DataFrame
data = np.hstack((X_test, Y_test.reshape(-1, 1), Y_test_pred.reshape(-1, 1)))
columns = [f"x{i+1}" for i in range(X_test.shape[1])] + ["y_test", "y_test_pred"]

# Create a DataFrame
df = pd.DataFrame(data, columns=columns)

# Export the DataFrame to a CSV file
output_file = "FNN_predictions.csv"
df.to_csv(output_file, index=False)

print(f"Predictions exported to {output_file}")

168/168 [==============================] - 0s 1ms/step
Predictions exported to FNN_predictions.csv


In [17]:
y_train_pred_scaled = model.predict(X_train)
y_test_pred_scaled = model.predict(X_test)

168/168 [==============================] - 0s 1ms/step


In [18]:
y_train_pred = y_scaler.inverse_transform(y_train_pred_scaled)
y_test_pred = y_scaler.inverse_transform(y_test_pred_scaled)

In [19]:
mae_train = mean_absolute_error(y_train, y_train_pred)
mse_train = mean_squared_error(y_train, y_train_pred)
mape_train = np.mean(np.abs((y_train - y_train_pred) / y_train)) * 100
rmse_train = np.sqrt(mse_train)
rmspe_train = np.sqrt(np.mean(np.square((y_train - y_train_pred) / y_train))) * 100 
r2_train = r2_score(y_train, y_train_pred)

mae_test = mean_absolute_error(y_test, y_test_pred)
mape_test = np.mean(np.abs((y_test - y_test_pred) / y_test)) * 100
mse_test = mean_squared_error(y_test, y_test_pred)
rmse_test = np.sqrt(mse_test)
rmspe_test = np.sqrt(np.mean(np.square((y_test - y_test_pred) / y_test))) * 100 
r2_test = r2_score(y_test, y_test_pred)

# Print results
print(f"Training Metrics:")
print(f"MAE: {mae_train:.4f}, MAPE: {mape_train:.4f}, MSE: {mse_train:.4f}, RMSE: {rmse_train:.4f}, RMSPE: {rmspe_train:.4f}, R2: {r2_train:.4f}")

print(f"Test Metrics:")
print(f"MAE: {mae_test:.4f}, MAPE: {mape_test:.4f}, MSE: {mse_test:.4f}, RMSE: {rmse_test:.4f}, RMSPE: {rmspe_test:.4f}, R2: {r2_test:.4f}")

Training Metrics:
MAE: 31.7121, MAPE: 2.3992, MSE: 2768.4197, RMSE: 52.6158, RMSPE: 4.6512, R2: 0.9986
Test Metrics:
MAE: 30.7347, MAPE: 2.1062, MSE: 2376.1755, RMSE: 48.7460, RMSPE: 3.1870, R2: 0.9986
